## Business Problem Description

- Customer churn data set of a bank that has 10k rows

- The bank has been seen unusual churn rates and they want to predict whether the customer has a risk to churn

In [ ]:
# Import packages
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
# Check the version of installed tensorflow package
tf.__version__

'2.7.0'

In [ ]:
# Import churn data set
df = pd.read_csv("/content/drive/MyDrive/Personal Project/Udemy - Churn Modeling ANN/Churn_Modelling.csv")
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


Additional information about the columns of the data:

- Tenure: how long customer have been with the bank
- Exited: left the bank (1) or stayed (0) -> Binary outcome

Create a geographic segmentation model to tell the bank which of the customer at the highest risk of leaving

In [ ]:
# Check if there is missing data
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

There are no missing values in the data set

## Data Preprocessing

In [ ]:
# Create a separate variable to store the independent and dependent variable
X = df.iloc[:, 3:-1]
y = df.iloc[:, -1]

In [ ]:
# Check the first five rows of the independent variables
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [ ]:
# Check the values of dependent variables
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [ ]:
# Get the information of the data type from the independent variables data set
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
dtypes: float64(2), int64(6), object(2)
memory usage: 781.4+ KB


We need to preprocess the geography and gender columns

In [ ]:
# Label encoding for gender variable
le = LabelEncoder()
X['Gender'] = le.fit_transform(X['Gender'])
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.80,3,1,0,113931.57
3,699,France,0,39,1,0.00,2,0,0,93826.63
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10


In [ ]:
# One hot encoding for geography variable
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)
X = np.array(X)

In [ ]:
# Split the data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# Feature (independent variables) standardization
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the ANN

In [ ]:
# Initializing ann as the sequence of layers
ann = tf.keras.models.Sequential()

# Add the input layer and the first hidden layer (fully connected layer)
# 'relu' refer to rectifier activation function
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

# Add the second hidden layer
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

# Add the output layer
# 'sigmoid' refer to sigmoid activation function
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Training the ANN

In [ ]:
# Compile the ANN
# 'binary_crossentropy' used spesifically to calculate loss for binary values of the dependent variable (target variable)
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Train ANN
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 3s 3ms/step - loss: 0.5050 - accuracy: 0.7937
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4594 - accuracy: 0.8001
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4391 - accuracy: 0.8076
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4288 - accuracy: 0.8163
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4236 - accuracy: 0.8171
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4206 - accuracy: 0.8198
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4183 - accuracy: 0.8214
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4166 - accuracy: 0.8227
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4151 - accuracy: 0.8246
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4134 - accura

## Simulating Prediction on a Single Observation

Should we say goodbye to the customer that has the following characteristics:

- Geography: France
- Credit Score: 600
- Gender: Male
- Age: 40 years old
- Tenure: 3 years
- Balance: \$ 60000
- Number of Products: 2
- Does this customer have a credit card ? Yes
- Is this customer an Active Member: Yes
- Estimated Salary: \$ 50000

In [ ]:
# Make sure that the feature values has been structured to the same format as the training set
ann.predict([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]) > 0.5

array([[ True]])

## Predicting the Test Set Result

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.to_numpy().reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [ ]:
# Making the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1472,  123],
       [ 237,  168]])

In [ ]:
# Calculate the accuracy score
accuracy_score(y_test, y_pred)

0.82

The ANN model has successfully predicted customer churn with 82% accuracy.